In [3]:
# 2. Reduction of the Discretization bias
# (a) give an estimate of the UOP option price with 95% CI
import numpy as np

def prob_cross_barrier(t_i, t_j, x, y, sig, H):
    prob = np.exp( (2/(sig*sig))/(t_j-t_i) * np.log(H/x) * np.log(y/H)) * (x<H)*(y<H) + (x>=H) + (y>=H) # True(1) + True(1) = True(1)
    return prob

prob_cross_barrier(t_i=0, t_j=1, x=80, y=85, sig=0.2, H=90)

np.random.seed(1)
N=int(5*1e5)

T = 0.25  # 3 months
M = 63 # 3개월을 데일리 샘플링
dt = T/M

# St = GBM
S0 = 50; sig=0.3; r=0.05; K=60; H=55; 

W_T = np.random.randn(N)*np.sqrt(T)
S_T = S0*np.exp((r-sig*sig/2)*T+sig*W_T)

prob_vec = prob_cross_barrier(0, T, S0, S_T, sig, H)

UOP_T = (K-S_T)*(K>=S_T)*(1-prob_vec)
UOP_0 = np.exp(-r*T)*UOP_T

print("95% CI = [", np.mean(UOP_0)-1.96*np.std(UOP_0)/np.sqrt(N), \
     ", ", np.mean(UOP_0)+1.96**np.std(UOP_0)/np.sqrt(N), "]")






95% CI = [ 6.842528842818334 ,  6.991517152199645 ]


In [6]:
# 2-(b) with Rebate

import numpy as np

def prob_cross_barrier(dt, x, y, sig, H):
    prob = np.exp( (2/(sig*sig))/(dt) * np.log(H/x) * np.log(y/H)) * (x<H)*(y<H) + (x>=H) + (y>=H) # True(1) + True(1) = True(1)
    return prob

np.random.seed(1)
N=int(5*1e5)

T = 0.25  # 3 months
M = 63 # 3개월을 데일리 샘플링
dt = T/M

# St = GBM
S0 = 50; sig=0.3; r=0.05; K=60; H=55; # 기대수익률(mu) 15%의 상품, 변동성(sig, 표준편차) 20%

# rebate
rebate = 5

# W, t = BM
W = np.zeros((N,M)) #  matrix
St = np.zeros((N,M))
t = np.zeros(M) 

# 시나리오 별(N) 변수(벡터) 추가
hitting_idx = np.zeros(N) # 처음에 50부터 시작이니 zero로 채움
first_hitting_time = np.zeros(N)*T # first hitting만 기록
S_prev = S0*np.ones(N) # 어제 종가

for j in range(M):
    dW = np.random.randn(N)*np.sqrt(dt) # 분산이 sqrt(dt)인 정규분포 Y
    
    if j<1:
        W[:,j] = dW
    else:
        W[:,j] = W[:,j-1] + dW
        
    t[j] = dt*(j+1)
    St[:,j] = S0*np.exp((r-sig*sig/2)*t[j]+sig*W[:,j]) # St의 모든열 중 i+1번째 행에 S0*exp를 대입
    
    hitting_prob = prob_cross_barrier(dt, S_prev, St[:,j], sig, H)
    unif_vec = np.random.rand(N)
    
    first_hitting_time = (t[j]-0.5*dt)*(1-hitting_idx)*(unif_vec<=hitting_prob) + first_hitting_time*hitting_idx # (중요) hitting idx = 1 경우 + 기존 hitting idx = 0 & 지금 생성된 주식가격 > H 일때 기록
    hitting_idx = (1-hitting_idx)*(unif_vec<=hitting_prob) + hitting_idx # (중요) hitting idx 업데이트, barrier 안쳤을 확률(hitting_prob) 체크
    S_prev = St[:,j]
    
#UOP_T = (K - St[:,-1])*(St[:,-1]<K)*(St_max<H)
UOP_0 = np.exp(-r*T)*(K-St[:,-1])*(St[:,-1]<K)*(1-hitting_idx) + np.exp(-r*first_hitting_time)*rebate*hitting_idx       # 만기까지 가서 pay-off 받거나, 중간에 쳐서 rebate 받거나
        

print("95% CI = [", np.mean(UOP_0)-1.96*np.std(UOP_0)/np.sqrt(N), \
     ", ", np.mean(UOP_0)+1.96**np.std(UOP_0)/np.sqrt(N), "]")

95% CI = [ 9.472670730085742 ,  9.553952489637718 ]


In [7]:
# 3. Barrier options under the CEV model
# (a) no rebate

import numpy as np

def prob_cross_barrier(t_i, t_j, x, y, sig, H):
    prob = np.exp( (2/(sig*sig))/(t_j-t_i) * np.log(H/x) * np.log(y/H)) * (x<H)*(y<H) + (x>=H) + (y>=H) # True(1) + True(1) = True(1)
    return prob

#prob_cross_barrier(t_i=0, t_j=1, x=80, y=85, sig=0.2, H=90)

np.random.seed(1)
N=int(5*1e5)

T = 0.25  # 3 months
M = 63 # 3개월을 데일리 샘플링
dt = T/M

# 
beta = -0.8
S0 = 50
alpha = 0.3 * (S0**(-beta))

# blah...blah (to be updated...)


95% CI = [ 9.440516289874488 ,  9.53200428599312 ]


In [8]:
# 3. Barrier options under the CEV model
# (b) with Rebate

import numpy as np

def prob_cross_barrier(dt, x, y, sig, H):
    prob = np.exp( (2/(sig*sig))/(dt) * np.log(H/x) * np.log(y/H)) * (x<H)*(y<H) + (x>=H) + (y>=H) # True(1) + True(1) = True(1)
    return prob

np.random.seed(1)
N=int(5*1e5)

T = 0.25  # 3 months
M = 63 # 3개월을 데일리 샘플링
dt = T/M

# St = GBM
S0 = 50; sig=0.3; r=0.05; K=60; H=55; # 기대수익률(mu) 15%의 상품, 변동성(sig, 표준편차) 20%

# rebate
rebate = 5

# W, t = BM
W = np.zeros((N,M)) #  matrix
St = np.zeros((N,M))
t = np.zeros(M) 

#
beta = -0.8
S0 = 50
alpha = 0.3 * (S0**(-beta))

# 시나리오 별(N) 변수(벡터) 추가
hitting_idx = np.zeros(N) # 처음에 50부터 시작이니 zero로 채움
first_hitting_time = np.zeros(N)*T # first hitting만 기록
S_prev = S0*np.ones(N) # 어제 종가

for j in range(M):
    dW = np.random.randn(N)*np.sqrt(dt) # 분산이 sqrt(dt)인 정규분포 Y
    
    if j<1:
        W[:,j] = dW
    else:
        W[:,j] = W[:,j-1] + dW
        
    t[j] = dt*(j+1)
    St[:,j] = S_prev + r*S_prev*dt + alpha*S_prev**(beta+1)*dW # 이 부분 핵심 !!!  
    
    hitting_prob = prob_cross_barrier(dt, S_prev, St[:,j], sig, H)
    unif_vec = np.random.rand(N)
    
    first_hitting_time = (t[j]-0.5*dt)*(1-hitting_idx)*(unif_vec<=hitting_prob) + first_hitting_time*hitting_idx # (중요) hitting idx = 1 경우 + 기존 hitting idx = 0 & 지금 생성된 주식가격 > H 일때 기록
    hitting_idx = (1-hitting_idx)*(unif_vec<=hitting_prob) + hitting_idx # (중요) hitting idx 업데이트, barrier 안쳤을 확률(hitting_prob) 체크
    S_prev = St[:,j]
    
#UOP_T = (K - St[:,-1])*(St[:,-1]<K)*(St_max<H)
UOP_0 = np.exp(-r*T)*(K-St[:,-1])*(St[:,-1]<K)*(1-hitting_idx) + np.exp(-r*first_hitting_time)*rebate*hitting_idx       # 만기까지 가서 pay-off 받거나, 중간에 쳐서 rebate 받거나
        

print("95% CI = [", np.mean(UOP_0)-1.96*np.std(UOP_0)/np.sqrt(N), \
     ", ", np.mean(UOP_0)+1.96**np.std(UOP_0)/np.sqrt(N), "]")

95% CI = [ 9.440516289874488 ,  9.53200428599312 ]
